In [24]:
import pandas as pd
import numpy as np
import struct
import re

In [25]:
# file = pd.read_csv("pingtestdecode.txt", sep='-')
file = open("test2.txt",'r')
# firstCol = np.asarray(file[0])
# print(firstCol)

a = file.read()

a.find('7e')

# get indices
indices = [m.start() for m in re.finditer('7e', a)]

packet_list = []
for index in range(len(indices) - 1):
    packet = a[indices[index]:indices[index+1] + 2]
    b = packet.split(sep='-')
    for i in range(len(b)):
        b[i]  = int('0x' + b[i],16)
    packet_list.append(b)
    
packet_list[0]

[126, 0, 1, 4, 0, 0, 0, 0, 116, 170, 126]

In [26]:
# CRC-CCITT calculation function based on the ICD
Crc16Table = [0] * 256

def crc_init():
    for i in range(256):
        crc = (i << 8)
        for bitctr in range(8):
            crc = (crc << 1) ^ (0x1021 if (crc & 0x8000) else 0)
        Crc16Table[i] = crc & 0xFFFF

def crc_compute(data):
    crc = 0
    for byte in data:
        crc = (Crc16Table[(crc >> 8) ^ byte] ^ (crc << 8)) & 0xFFFF
    return crc

# Unescape the control bytes
def unescape_payload(payload):
    unescaped = bytearray()
    i = 0
    while i < len(payload):
        if payload[i] == 0x7D:  # Control escape
            i += 1
            unescaped.append(payload[i] ^ 0x20)  # XOR next byte with 0x20
        else:
            unescaped.append(payload[i])
        i += 1
    return unescaped

# Decode the packet based on message ID
def decode_packet(packet):
    if len(packet) < 3:
        return "Invalid packet length"

    # Extract message ID and version (assume message ID is 1 byte, adjust as needed)
    msg_id = packet[0]
    version = packet[1]

    # Example: Handling Heartbeat message (Msg ID = 0x00)
    if msg_id == 0x00:  # Heartbeat
        gnss_valid = (packet[2] >> 7) & 0x01
        maintenance_req = (packet[2] >> 6) & 0x01
        ident_active = (packet[2] >> 5) & 0x01
        initialized = packet[2] & 0x01

        return {
            "Message ID": msg_id,
            "GNSS Valid": gnss_valid,
            "Maintenance Required": maintenance_req,
            "IDENT Active": ident_active,
            "Device Initialized": initialized
        }

    # Add handling for more messages (e.g., Ownship, GNSS Data) using the format from the ICD
    return msg_id, None

# Process the input packet
def process_packet(data):
    if data[0] != 0x7E or data[-1] != 0x7E:
        return "Invalid frame flags"
    
    # Remove framing flags (0x7E at start and end)
    packet = data[1:-1]

    # Unescape payload
    packet = unescape_payload(packet)

    # Verify CRC
    payload, crc_received = packet[:-2], packet[-2:]
    crc_computed = crc_compute(payload)
    crc_received = struct.unpack('>H', crc_received)[0]

    # print(crc_computed, crc_received)
    
    # if crc_computed != crc_received:
    #     return "CRC check failed"

    # Decode payload
    return decode_packet(payload)

# Test the script with a sample packet
# sample_packet = bytearray([0x7E, 0x80, 0x60, 0x18, 0x80, 0x80, 0x80, 0xFE, 0xFF, 0xFF, 0xFF, 0xFB, 0x30, 0x18, 0x7B, 0xFC, 0x7E])  # Example with escaped data
sample_packet = bytearray(packet_list[6])
crc_init()  # Initialize CRC table
result = process_packet(sample_packet)
print(result)
for packet in packet_list:
    try:
        sample_packet = bytearray(packet)
        result = process_packet(sample_packet)
        print(result)
    except:
        print('Error')

(47, None)
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
Error
(37, None)
Error
(10, None)
Error
(47, None)
Error
(40, None)
Error
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
Error
(37, None)
Error
(10, None)
Error
(47, None)
Error
(40, None)
Error
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
Error
(37, None)
Error
(10, None)
Error
(47, None)
Error
(40, None)
Error
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
Error
(37, None)
Error
(10, None)
Error
(47, None)
Error
(40, None)
Error
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
Error
(37, None)
Error
(10, None)
Error
(47, None)
Error
(40, None)
Error
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device 